In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import shutil
import seaborn as sns
import os
from itertools import combinations



def stats_plot(experiment_name, stats, path, gen_set, exp_key,seeds):
    labels = ['A','B']
    fig, ax = plt.subplots(1, 3, figsize=(20, 6))
    c=0
    for seed in seeds:        
        stats_df = pd.DataFrame(stats, columns=['generations', 'median', 'best'])
        
        #print(stats_df)
        sns.lineplot(data=stats_df, x="generations", y="best", label=exp_key[0]+"Best", ax=ax[0])
        sns.lineplot(data=stats_df, x="generations", y="median", label=exp_key[0] + "Median", ax=ax[0])
        ax[c].set_xlabel('Generations')
        ax[c].set_ylabel('Fitness')
        ax[c].set_xlim(0, 1000)
        ax[c].set_ylim(0, 1)
        ax[c].set_title(labels[c], loc='left')
        ax[c].legend(loc='upper left')
        c+=1
        break

    plt.savefig(path+'stats.pdf', bbox_inches='tight')

def generate_plots(generations, niches, gen_set,runs,exp_name):
    matplotlib.rcParams.update({'font.size': 14})
    exp_key = ['']
    path = './plots/gen_'+str(generations[0])+'_'+str(generations[1])+'_niches_'+str(niches)+'/'
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)
    coverage = []
    stats = []
    alt_env_overall = []
    exp_data = []
    seeds=[]
    with open("seed.txt") as file:
        seeds = [line.rstrip() for line in file]
    
    
    temp_stats = []
    folder = './' + exp_name + "_gen_" + str(1000) + "_" + str(generations[1]) + "_niche_" + str(niches) + "_"
    
    tempmax={}
    newseeds=[]
    for seed in seeds:
        data = pd.read_csv(folder+'run'+str(seed)+'/layer3_archive_'+str(1000)+'.dat', sep=" ", header=None)
        data.columns = ["ID", "Fitness", "Friction", "Restitution", "Size", "rem"]
        tempmax[seed]=np.max(data["Fitness"])
    print(tempmax)
    tempmax = dict(sorted(tempmax.items(), key=lambda item: item[1],reverse = True)[:10])
    for key in tempmax:
        newseeds.append(key)   
    print(seeds)
    print(newseeds)
    for seed in seeds:
        for i in range(0, 1000+1, 10):
            k = i
            #and (seed == "15296" or seed == "62232")
            if i>= 1190 and i < 1210:
                k = 1190
                data = pd.read_csv(folder+'run'+str(seed)+'/layer3_archive_'+str(k)+'.dat', sep=" ", header=None)
                data.columns = ["ID", "Fitness", "Friction", "Restitution", "Size", "rem"]
            
                temp_stats.append([i, np.median(data["Fitness"]/10), np.max(data["Fitness"]/10)])
            else:
                data = pd.read_csv(folder+'run'+str(seed)+'/layer3_archive_'+str(k)+'.dat', sep=" ", header=None)
                data.columns = ["ID", "Fitness", "Friction", "Restitution", "Size", "rem"]
            
                temp_stats.append([i, np.median(data["Fitness"]/10), np.max(data["Fitness"])/10])

    
    stats_plot(exp_name, temp_stats, path, gen_set, exp_key,seeds)



                     
if __name__ == '__main__':
    all_tests = [7200,4800,3600]
    level_gens = [1000, 3000]
    level_niche = 1000
    runs = 10
    exp_name = "sim3600moregen"
    generate_plots(level_gens, level_niche, all_tests,runs,exp_name)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import shutil
import seaborn as sns
import os
from itertools import combinations
pd.options.mode.chained_assignment = None

with open("seed.txt") as file:
    seeds = [line.rstrip() for line in file]

generations = [1000, 3000]
niches = 1000
exp_name = "sim3600moregen"

folder = './' + exp_name + "_gen_" + str(1000) + "_" + str(generations[1]) + "_niche_" + str(niches) + "_"

k=1000
new_df = pd.DataFrame(columns=["ID", "Fitness", "Friction", "Restitution", "Size", "rem",'Seed'])
for seed in seeds:
    data = pd.read_csv(folder+'run'+str(seed)+'/layer3_archive_'+str(k)+'.dat', sep=" ", header=None)
    data.columns = ["ID", "Fitness", "Friction", "Restitution", "Size", "rem"]

    max_fitness_idx = data['Fitness'].idxmax()

    row_with_max_fitness = data.loc[max_fitness_idx]

    row_with_max_fitness['Seed'] = seed
    new_df = new_df.append(row_with_max_fitness, ignore_index=True)

max_fitness_idx = new_df['Fitness'].idxmax()
row_with_max_fitness = new_df.loc[max_fitness_idx]
print(new_df)